# Install Dependencies
Install additional packages needed to run SQL command in Jupyter cells and send them to a Postgres database server.

## PostgreSQL
Allows for a self-contained instance of PostresSQL for this notebook.

In [ ]:
%mamba install -y -c conda-forge postgresql=17 --quiet
!psql --version

## PostgreSQL Adapter for Python
Essential for connecting and interacting with PostgreSQL databases from Python code in Jupyter notebooks.

In [ ]:
%mamba install -c conda-forge psycopg2 -y --quiet
!python -c "import psycopg2; print(psycopg2.__version__)"

## Language Server Protocol for Jupyter
Enables code completion, hover documentation, and real-time syntax checking for multiple languages in Jupyter notebooks, including SQL.

In [ ]:
%mamba install -c conda-forge jupyterlab-lsp -y --quiet
!jupyter labextension list
!jupyter server extension list

## Language Server for Python
Provides real-time linting, code suggestions, and auto-completion in Jupyter notebooks for Python code.

In [ ]:
%mamba install -c conda-forge python-lsp-server -y --quiet
!pylsp --version

## SQL Magic Extension for Jupyter
Allows running SQL queries directly in Jupyter notebooks using SQL magic (%%sql). Enables seamless execution of SQL queries inside a Jupyter Notebook without switching to a database client.

In [ ]:
%mamba install -c conda-forge jupysql -y --quiet
!python -c "import sql; print(sql.__version__)"
!jupyter labextension list

## SQL Editor Extension
Enables SQL-specific features in Jupyter notebooks, such as:
* Syntax highlighting
* Query auto-completion
* Error checking

In [ ]:
%pip install jupyterlab_sql_editor --quiet
!jupyter labextension list

# Start PostgreSQL

## Define paths and logfile

In [ ]:
import os

DB_DIR = "mylocal_db"
LOGFILE = "postgres_log.txt"
PG_HBA_PATH = os.path.join(DB_DIR, "pg_hba.conf")

## Check PostgreSQL status

In [ ]:
!pg_ctl -D $DB_DIR status

## Stop any existing PostgreSQL process

In [ ]:
!pg_ctl stop -D $DB_DIR -m fast || echo "Nothing to stop."

## Initialize the database if necessary

In [ ]:
import os

if not os.path.exists(DB_DIR) or not os.path.exists(os.path.join(DB_DIR, "PG_VERSION")):
    os.makedirs(DB_DIR, exist_ok=True)
    !initdb -D $DB_DIR

## Modify pg_hba.conf to allow trust authentication for user "notebook"

In [ ]:
with open(PG_HBA_PATH, "a") as pg_hba:
    pg_hba.write("\nlocal   all             notebook                                trust\n")
    pg_hba.write("host    all             notebook        127.0.0.1/32          trust\n")
    pg_hba.write("host    all             notebook        ::1/128               trust\n")

## Start PostgreSQL server

In [ ]:
import subprocess
import time
import atexit

def stop_postgres():
    !pg_ctl -D $DB_DIR stop

atexit.register(stop_postgres)

with open(LOGFILE, "a") as logfile:
    subprocess.Popen(["pg_ctl", "-D", DB_DIR, "-l", LOGFILE, "start"],
                     stdout=logfile, stderr=logfile)
time.sleep(5)

## Create the "notebook" user with full permissions

In [ ]:
!psql -d postgres -c "CREATE USER notebook WITH SUPERUSER;" || echo "User already exists."
!psql -d postgres -c "ALTER USER notebook WITH PASSWORD NULL;"

# Connect

## Load & Configure SQLMagic

In [ ]:
%load_ext sql
%config SqlMagic.displaylimit = None

## Create Fresh sahuagin Database

In [ ]:
%sql postgresql://notebook@localhost:5432/postgres
%sql SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE pg_stat_activity.datname = 'sahuagin';
%sql DROP DATABASE IF EXISTS sahuagin;
%sql CREATE DATABASE sahuagin;

## Connect to sahuagin Database

In [ ]:
%sql postgresql://notebook@localhost:5432/sahuagin

# Create Tables & Types

## Attribute Type

In [ ]:
%%sql

DROP TYPE IF EXISTS attribute_type CASCADE;
CREATE TYPE attribute_type AS ENUM ('discrete','continuous');

## Attribute

In [ ]:
%%sql

DROP TABLE IF EXISTS attribute CASCADE;
CREATE TABLE attribute (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE,
    type attribute_type NOT NULL,
    decimals integer,
    has_labels boolean,
    has_value boolean,
    max_value double precision,
    min_value double precision,
    mode_value double precision,
    concentration double precision,
    skew double precision,
    units varchar(255),
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND decimals IS NULL
            AND has_labels IS NULL
            AND has_value IS NULL
            AND max_value IS NULL
            AND min_value IS NULL
            AND mode_value IS NULL
            AND concentration IS NULL
            AND skew IS NULL
            AND units IS NULL
        )
        OR (
            type = 'continuous'
            AND decimals IS NOT NULL
            AND has_labels IS NOT NULL
            AND has_value IS NOT NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
            AND mode_value IS NOT NULL
            AND concentration IS NOT NULL
            AND skew IS NOT NULL
        )
    )
);

## Span

In [ ]:
%%sql

DROP TABLE IF EXISTS span CASCADE;
CREATE TABLE span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    label varchar(255) NOT NULL,
    type attribute_type NOT NULL,
    is_percentage_pinned boolean,
    weight double precision,
    max_value double precision,
    min_value double precision,
    CONSTRAINT unique_attribute_type_label UNIQUE (attribute_id, type, label),
    CONSTRAINT fk_spans_attr FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CHECK (
        max_value IS NULL
        OR min_value IS NULL
        OR max_value >= min_value
    ),
    CHECK (
        (
            type = 'discrete'
            AND is_percentage_pinned IS NOT NULL
            AND weight IS NOT NULL
            AND max_value IS NULL
            AND min_value IS NULL
        )
        OR (
            type = 'continuous'
            AND is_percentage_pinned IS NULL
            AND weight IS NULL
            AND max_value IS NOT NULL
            AND min_value IS NOT NULL
        )
    )
);

## Variant

In [ ]:
%%sql

DROP TABLE IF EXISTS variant CASCADE;
CREATE TABLE variant (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

## Variant Attribute

In [ ]:
%%sql

DROP TABLE IF EXISTS variant_attribute CASCADE;
CREATE TABLE variant_attribute (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    attribute_id integer NOT NULL,
    name varchar(255) NOT NULL,
    causation_index integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_name UNIQUE (variant_id, name),
    CONSTRAINT unique_variant_index UNIQUE (variant_id, causation_index),
    CONSTRAINT fk_variant_attr_attr FOREIGN KEY (attribute_id) REFERENCES attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_variant_attr_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

## Sub-Variant Activating Span

In [ ]:
%%sql

DROP TABLE IF EXISTS variant_attr_span CASCADE;
CREATE TABLE variant_attr_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    span_id integer NOT NULL,
    variant_attribute_id integer NOT NULL,
    variant_id integer NOT NULL,
    CONSTRAINT unique_variant_attribute_span UNIQUE (variant_id, variant_attribute_id, span_id),
    CONSTRAINT fk_varattr_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_varattr_span_varattr FOREIGN KEY (variant_attribute_id) REFERENCES variant_attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_varattr_span_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

## Variant Attribute Reference

In [ ]:
%%sql

DROP TABLE IF EXISTS vavspan_attr CASCADE;
CREATE TABLE vavspan_attr (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_attribute_id integer NOT NULL,
    variant_attr_span_id integer,
    CONSTRAINT unique_variant_attribute UNIQUE (variant_attr_span_id, variant_attribute_id),
    CONSTRAINT fk_vavspan_attr_va FOREIGN KEY (variant_attribute_id) REFERENCES variant_attribute (id) ON DELETE CASCADE,
    CONSTRAINT fk_vavspan_attr_vas FOREIGN KEY (variant_attr_span_id) REFERENCES variant_attr_span (id) ON DELETE CASCADE
);

## Variation

In [ ]:
%%sql

DROP TABLE IF EXISTS variation CASCADE;
CREATE TABLE variation (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    activating_span_id integer NOT NULL,
    to_modify_vavspan_attr_id integer NOT NULL,
    activating_vavspan_attr_id integer NOT NULL,
    is_inactive boolean NOT NULL DEFAULT false,
    CONSTRAINT unique_span_to_attribute UNIQUE (activating_span_id, activating_vavspan_attr_id, to_modify_vavspan_attr_id),
    CONSTRAINT fk_variation_span FOREIGN KEY (activating_span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_variation_vavspan_to_modify FOREIGN KEY (to_modify_vavspan_attr_id) REFERENCES vavspan_attr (id) ON DELETE CASCADE,
    CONSTRAINT fk_variation_vavspan_activating FOREIGN KEY (activating_vavspan_attr_id) REFERENCES vavspan_attr (id) ON DELETE CASCADE
);

## Delta Parameter Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_continuous_attr CASCADE;
CREATE TABLE variation_continuous_attr (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variation_id integer NOT NULL,
    delta_mode double precision NOT NULL,
    delta_conc double precision NOT NULL,
    delta_skew double precision NOT NULL,
    CONSTRAINT fk_var_continuous_attr_var FOREIGN KEY (variation_id) REFERENCES variation (id) ON DELETE CASCADE
);

## Additional Possibility Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_activated_span CASCADE;
CREATE TABLE variation_activated_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    span_id integer NOT NULL,
    variation_id integer NOT NULL,
    CONSTRAINT unique_active_variation_span UNIQUE (variation_id, span_id),
    CONSTRAINT fk_var_activated_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_activated_span_var FOREIGN KEY (variation_id) REFERENCES variation (id) ON DELETE CASCADE
);

## Delta Weight Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_delta_weight CASCADE;
CREATE TABLE variation_delta_weight (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    delta_weight double precision NOT NULL,
    span_id integer NOT NULL,
    variation_id integer NOT NULL,
    CONSTRAINT fk_var_delta_weight_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_delta_weight_var FOREIGN KEY (variation_id) REFERENCES variation (id) ON DELETE CASCADE
);

## Removed Possibility Effect

In [ ]:
%%sql

DROP TABLE IF EXISTS variation_inactive_span CASCADE;
CREATE TABLE variation_inactive_span (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    span_id integer NOT NULL,
    variation_id integer NOT NULL,
    CONSTRAINT unique_inactive_variation_span UNIQUE (variation_id, span_id),
    CONSTRAINT fk_var_inactive_span_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_var_inactive_span_var FOREIGN KEY (variation_id) REFERENCES variation (id) ON DELETE CASCADE
);

## Entity

In [ ]:
%%sql

DROP TABLE IF EXISTS entity CASCADE;
CREATE TABLE entity (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    variant_id integer NOT NULL,
    name varchar(255) NOT NULL UNIQUE,
    CONSTRAINT fk_entity_variant FOREIGN KEY (variant_id) REFERENCES variant (id) ON DELETE CASCADE
);

## Entity Group

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_group CASCADE;
CREATE TABLE entity_group (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    name varchar(255) NOT NULL UNIQUE
);

## Entity Group Link

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_group_link CASCADE;
CREATE TABLE entity_group_link (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    entity_group_id integer NOT NULL,
    CONSTRAINT unique_link_entity_group UNIQUE (entity_group_id, entity_id),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE,
    CONSTRAINT fk_entity_group FOREIGN KEY (entity_group_id) REFERENCES entity_group (id) ON DELETE CASCADE
);

## Entity State

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_state CASCADE;
CREATE TABLE entity_state (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_id integer NOT NULL,
    time double precision NOT NULL,
    CONSTRAINT unique_entity_time UNIQUE (entity_id, time),
    CONSTRAINT fk_entity FOREIGN KEY (entity_id) REFERENCES entity (id) ON DELETE CASCADE
);

## Attribute Value

In [ ]:
%%sql

DROP TABLE IF EXISTS entity_varattr_value CASCADE;
CREATE TABLE entity_varattr_value (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    entity_state_id integer NOT NULL,
    numeric_value double precision,
    span_id integer,
    vavspan_attr_id integer NOT NULL,
    CONSTRAINT unique_state_vavspan_attr UNIQUE (entity_state_id, vavspan_attr_id),
    CONSTRAINT fk_evav_entity_state FOREIGN KEY (entity_state_id) REFERENCES entity_state (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_span FOREIGN KEY (span_id) REFERENCES span (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_vavspan_attr FOREIGN KEY (vavspan_attr_id) REFERENCES vavspan_attr (id) ON DELETE CASCADE
);

## Attribute Value Lock

In [ ]:
%%sql

DROP TABLE IF EXISTS evav_lock CASCADE;
CREATE TABLE evav_lock (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    locked_evav_id integer NOT NULL,
    locking_evav_id integer,
    CONSTRAINT unique_evav_lock UNIQUE (locking_evav_id, locked_evav_id),
    CONSTRAINT fk_evav_lock_locked_evav FOREIGN KEY (locked_evav_id) REFERENCES entity_varattr_value (id) ON DELETE CASCADE,
    CONSTRAINT fk_evav_lock_locking_evav FOREIGN KEY (locking_evav_id) REFERENCES entity_varattr_value (id) ON DELETE CASCADE
);

# Create Indexes

## idx_attribute_name

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_attribute_name;
CREATE INDEX idx_attribute_name ON attribute (name);

## idx_span_attribute_id

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_span_attribute_id;
CREATE INDEX idx_span_attribute_id ON span (attribute_id);

## idx_span_attribute_type

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_span_attribute_type;
CREATE INDEX idx_span_attribute_type ON span(attribute_id, type, id);

## idx_span_attr_type

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type;
CREATE INDEX idx_span_attr_type ON span(attribute_id, type, min_value, max_value);

## idx_span_attr_type_pinned

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned;
CREATE INDEX idx_span_attr_type_pinned ON span(attribute_id, type, is_percentage_pinned);

## idx_span_attr_type_pinned_wl

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_span_attr_type_pinned_wl;
CREATE INDEX idx_span_attr_type_pinned_wl ON span(attribute_id, type, is_percentage_pinned, weight, label);

## idx_variant_name

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_name;
CREATE INDEX idx_variant_name ON variant (name);

## idx_variant_attribute_attribute

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attribute_attribute;
CREATE INDEX idx_variant_attribute_attribute ON variant_attribute (attribute_id);

## idx_variant_attribute_variant

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attribute_variant;
CREATE INDEX idx_variant_attribute_variant ON variant_attribute (variant_id);

## idx_variant_attribute_variant_causation

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attribute_variant_causation;
CREATE INDEX idx_variant_attribute_variant_causation ON variant_attribute (variant_id, causation_index);

## idx_variant_attr_span_span_id

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_span_id;
CREATE INDEX idx_variant_attr_span_span_id ON variant_attr_span (span_id);

## idx_variant_attr_span_variant_attribute_id

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_variant_attribute_id;
CREATE INDEX idx_variant_attr_span_variant_attribute_id ON variant_attr_span (variant_attribute_id);

## idx_variant_attr_span_variant_attr

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_variant_attr;
CREATE INDEX idx_variant_attr_span_variant_attr ON variant_attr_span (
    variant_id,
    variant_attribute_id
);

## idx_variant_attr_span_vaid_spanid

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_vaid_spanid;
CREATE INDEX idx_variant_attr_span_vaid_spanid ON variant_attr_span (variant_attribute_id, span_id);

## idx_variant_attr_span_vaid_variant

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variant_attr_span_vaid_variant;
CREATE INDEX idx_variant_attr_span_vaid_variant ON variant_attr_span (
    variant_attribute_id,
    variant_id
);

## idx_vavspan_attr_variant_attribute

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_vavspan_attr_variant_attribute;
CREATE INDEX idx_vavspan_attr_variant_attribute ON vavspan_attr (variant_attribute_id);

## idx_vavspan_attr_vaid

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_vavspan_attr_vaid;
CREATE INDEX idx_vavspan_attr_vaid ON vavspan_attr (variant_attribute_id, id);

## idx_uq_vavspan_attr

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_uq_vavspan_attr;
CREATE INDEX idx_uq_vavspan_attr ON vavspan_attr (
    variant_attribute_id,
    variant_attr_span_id
);

## idx_var_cont_variation

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_var_cont_variation;
CREATE INDEX idx_var_cont_variation ON variation_continuous_attr (variation_id);

## idx_variation_activated_span_varid_spanid

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variation_activated_span_varid_spanid;
CREATE INDEX idx_variation_activated_span_varid_spanid ON variation_activated_span (variation_id, span_id);

## idx_variation_inactive

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variation_inactive;
CREATE INDEX idx_variation_inactive ON variation_inactive_span (variation_id, span_id);

## idx_variation_delta_weight_varid_spanid

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta_weight_varid_spanid;
CREATE INDEX idx_variation_delta_weight_varid_spanid ON variation_delta_weight (variation_id, span_id);

## idx_variation_delta

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variation_delta;
CREATE INDEX idx_variation_delta ON variation_delta_weight (
    variation_id,
    span_id,
    delta_weight
);

## idx_variation_is_inactive

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variation_is_inactive;
CREATE INDEX idx_variation_is_inactive ON variation (is_inactive, id);

## idx_variation_to_modify_inactive

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_variation_to_modify_inactive;
CREATE INDEX idx_variation_to_modify_inactive ON variation (
    to_modify_vavspan_attr_id,
    is_inactive
);

## idx_entity_state_entity_id

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_id;
CREATE INDEX idx_entity_state_entity_id ON entity_state (entity_id);

## idx_entity_state_entity_time

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_entity_state_entity_time;
CREATE INDEX idx_entity_state_entity_time ON entity_state (entity_id, time);

## idx_entity_varattr_value_entity_state_id

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_entity_varattr_value_entity_state_id;
CREATE INDEX idx_entity_varattr_value_entity_state_id ON entity_varattr_value (entity_state_id);

## idx_evav_state_vaid

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_evav_state_vaid;
CREATE INDEX idx_evav_state_vaid ON entity_varattr_value (
    entity_state_id,
    variant_attribute_id
);

## idx_evav_state_attr_span

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_evav_state_attr_span;
CREATE INDEX idx_evav_state_attr_span ON entity_varattr_value (
    entity_state_id,
    variant_attribute_id,
    span_id
);

## idx_evav_lock_locked

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locked;
CREATE INDEX idx_evav_lock_locked ON evav_lock (locked_evav_id);

## idx_evav_lock_locking

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locking;
CREATE INDEX idx_evav_lock_locking ON evav_lock (locking_evav_id);

## idx_evav_lock_locked_locking

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_evav_lock_locked_locking;
CREATE INDEX idx_evav_lock_locked_locking ON evav_lock (
    locked_evav_id,
    locking_evav_id
);

## idx_entity_group_link_group_id_entity_id

In [ ]:
%%sql

DROP INDEX IF EXISTS idx_entity_group_link_group_id_entity_id;
CREATE INDEX idx_entity_group_link_group_id_entity_id
  ON entity_group_link(entity_group_id, entity_id);

# Debug

## Create Log Table

In [ ]:
%%sql

DROP TABLE IF EXISTS debug_log CASCADE;
CREATE TABLE debug_log (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    log_time timestamp DEFAULT CURRENT_TIMESTAMP,
    procedure_name varchar(255),
    log_message text
);

## Clear Log Table

In [ ]:
%%sql

TRUNCATE TABLE debug_log RESTART IDENTITY CASCADE;

## Create Log Function

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION debug_log(
    p_procedure_name varchar,
    p_log_message text
) RETURNS void
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO debug_log (procedure_name, log_message, log_time)
    VALUES (p_procedure_name, p_log_message, now());
END;
$$;

# Create Functions & Procedures

## Add Discrete Attribute

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_discrete_attribute(
    in_name       VARCHAR(255)
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO attribute (name, type)
      VALUES (in_name, 'discrete');
END;
$$;

## Redistribute Unpinned Spans

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION redistribute_unpinned_spans(
    p_attribute_id INTEGER,
    p_modified_span_id INTEGER,
    p_new_weight DOUBLE PRECISION,
    p_old_weight DOUBLE PRECISION
) RETURNS VOID
LANGUAGE plpgsql
AS $$
DECLARE
    v_pinned_weight DOUBLE PRECISION;
    v_target         DOUBLE PRECISION;
BEGIN
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = true;
       
    -- The available weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    
    -- If there’s only one unpinned span (v_target equals p_old_weight), then
    -- the new weight must equal the entire target.
    IF (v_target - p_old_weight) = 0 THEN
       IF p_new_weight <> v_target THEN
           RAISE EXCEPTION 'Cannot change weight; only one unpinned span exists and its weight must be %', v_target;
       END IF;
    END IF;
    
    -- Update all unpinned discrete spans:
    -- • The modified span gets the new weight.
    -- • All others are scaled proportionally.
    UPDATE span
       SET weight = CASE 
                      WHEN id = p_modified_span_id THEN p_new_weight
                      ELSE weight * ((v_target - p_new_weight) / (v_target - p_old_weight))
                    END
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = false;
END;
$$;

## Add Discrete Span to Attribute

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_disc_span_to_attr(
    p_attribute_id INTEGER,
    p_label VARCHAR(255)
)
LANGUAGE plpgsql
AS $$
DECLARE
    new_span_id     INTEGER;
    v_total_weight  DOUBLE PRECISION;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_count_old     INTEGER;
    candidate       DOUBLE PRECISION;
BEGIN
    -- Compute the total weight for all discrete spans for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_total_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete';

    -- If no spans exist, simply insert the first span with weight 1.
    IF v_total_weight = 0 THEN
        INSERT INTO span(attribute_id, label, type, is_percentage_pinned, weight)
        VALUES (p_attribute_id, p_label, 'discrete', false, 1.0);
        RETURN;
    END IF;
    
    -- Compute the total weight of pinned discrete spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = true;
    
    -- Calculate the available target weight for all unpinned spans.
    v_target := 1.0 - v_pinned_weight;
    IF v_target <= 0 THEN
        RAISE EXCEPTION 'No available weight for unpinned spans (v_target = %)', v_target;
    END IF;
    
    -- Count the existing non-pinned discrete spans.
    SELECT COUNT(*)
      INTO v_count_old
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = false;
    
    -- Insert the new unpinned span with a temporary weight of 0.
    INSERT INTO span(attribute_id, label, type, is_percentage_pinned, weight)
    VALUES (p_attribute_id, p_label, 'discrete', false, 0.0)
    RETURNING id INTO new_span_id;
    
    -- Compute the candidate weight for the new span.
    candidate := v_target / (v_count_old + 1);
    
    -- Redistribute the weights among all unpinned spans
    PERFORM redistribute_unpinned_spans(p_attribute_id, new_span_id, candidate, 0);
END;
$$;

## Modify Discrete Span Weight

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE modify_disc_span_weight(
    p_span_id    INTEGER,
    p_new_weight DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id INTEGER;
    v_current_weight DOUBLE PRECISION;
    v_is_pinned    BOOLEAN;
    v_type         TEXT;
    v_pinned_weight DOUBLE PRECISION;
    v_target        DOUBLE PRECISION;
    v_other_sum     DOUBLE PRECISION;
BEGIN
    -- Get the current span's details.
    SELECT attribute_id, weight, is_percentage_pinned, type
      INTO v_attribute_id, v_current_weight, v_is_pinned, v_type
      FROM span
     WHERE id = p_span_id;
     
    -- Ensure the span exists, is of type 'discrete' and is not pinned.
    IF NOT FOUND OR v_type <> 'discrete' OR v_is_pinned THEN
       RAISE EXCEPTION 'Span % not found or not a modifiable discrete span', p_span_id;
    END IF;
    
    -- Compute the total pinned weight for this attribute.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_pinned_weight
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = true;
       
    v_target := 1.0 - v_pinned_weight;
    
    -- Validate the new weight is between 0 and the available target.
    IF p_new_weight < 0 OR p_new_weight > v_target THEN
       RAISE EXCEPTION 'Invalid new weight: % (must be between 0 and %)', p_new_weight, v_target;
    END IF;
    
    -- Compute the total weight for all other unpinned spans.
    SELECT COALESCE(SUM(weight), 0)
      INTO v_other_sum
      FROM span
     WHERE attribute_id = v_attribute_id
       AND type = 'discrete'
       AND is_percentage_pinned = false
       AND id <> p_span_id;
       
    IF v_other_sum = 0 THEN
       -- There is only one unpinned span. Its weight must equal the available target.
       IF p_new_weight <> v_target THEN
         RAISE EXCEPTION 'Only one unpinned span exists. Its weight must be %', v_target;
       ELSE
         UPDATE span SET weight = p_new_weight WHERE id = p_span_id;
       END IF;
    ELSE
       -- Use the helper function to redistribute weights.
       PERFORM redistribute_unpinned_spans(v_attribute_id, p_span_id, p_new_weight, v_current_weight);
    END IF;
END;
$$;

## Add Continuous Attribute

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_continuous_attribute(
    p_name           VARCHAR(255),
    p_min_value      DOUBLE PRECISION,
    p_mode_value     DOUBLE PRECISION,
    p_max_value      DOUBLE PRECISION,
    p_concentration  DOUBLE PRECISION,
    p_skew           DOUBLE PRECISION,
    p_decimals       INTEGER,
    p_has_labels     BOOLEAN,
    p_has_value      BOOLEAN,
    p_units          VARCHAR(255),
    OUT p_attr_id    INTEGER
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO attribute
      (name, type, decimals, has_labels, has_value, max_value, min_value,
       mode_value, concentration, skew, units)
    VALUES
      (p_name, 'continuous', p_decimals, p_has_labels, p_has_value, p_max_value,
       p_min_value, p_mode_value, p_concentration, p_skew, p_units)
    RETURNING id INTO p_attr_id;
END;
$$;

## Add Continuous Span to Attribute

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_cont_span_to_attr(
    p_attribute_id integer,
    p_label varchar(255),
    p_new_min double precision,
    p_new_max double precision
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attr_min double precision;
    v_attr_max double precision;
    v_count integer;
    v_first_id integer;
    v_first_min double precision;
    v_first_max double precision;
    v_last_id integer;
    v_last_min double precision;
    v_last_max double precision;
BEGIN
    -- Get attribute’s overall range (continuous only)
    SELECT min_value, max_value
      INTO v_attr_min, v_attr_max
      FROM attribute
     WHERE id = p_attribute_id
       AND type = 'continuous'
     LIMIT 1;
     
    IF v_attr_min IS NULL THEN
        RAISE EXCEPTION 'Attribute not found or not continuous';
    END IF;
    
    -- Validate requested span
    IF p_new_min < v_attr_min OR p_new_max > v_attr_max OR p_new_min >= p_new_max THEN
        RAISE EXCEPTION 'Invalid span range';
    END IF;
    
    -- If no continuous spans exist, insert one covering the full range
    SELECT COUNT(*) INTO v_count
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous';
       
    IF v_count = 0 THEN
        INSERT INTO span(attribute_id, label, type, min_value, max_value)
        VALUES(p_attribute_id, p_label, 'continuous', v_attr_min, v_attr_max);
        RETURN;
    END IF;
    
    -- Adjust overlapping spans: first overlapping span (ordered by min_value)
    SELECT id, min_value, max_value
      INTO v_first_id, v_first_min, v_first_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY min_value ASC
     LIMIT 1;
     
    IF v_first_min < p_new_min THEN
        UPDATE span SET max_value = p_new_min WHERE id = v_first_id;
    END IF;
    
    -- Adjust overlapping spans: last overlapping span (ordered by max_value)
    SELECT id, min_value, max_value
      INTO v_last_id, v_last_min, v_last_max
      FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND max_value > p_new_min
       AND min_value < p_new_max
     ORDER BY max_value DESC
     LIMIT 1;
     
    IF v_last_max > p_new_max THEN
        UPDATE span SET min_value = p_new_max WHERE id = v_last_id;
    END IF;
    
    -- Remove spans fully covered by the new span
    DELETE FROM span
     WHERE attribute_id = p_attribute_id
       AND type = 'continuous'
       AND min_value >= p_new_min
       AND max_value <= p_new_max;
       
    INSERT INTO span(attribute_id, label, type, min_value, max_value)
    VALUES(p_attribute_id, p_label, 'continuous', p_new_min, p_new_max);
END;
$$;

## Link Attribute to Variant

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE link_attr_to_variant(
    p_variant_id   INTEGER,
    p_attribute_id INTEGER,
    p_name         VARCHAR,
    p_position     INTEGER
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_count    INTEGER;
    v_position INTEGER;
BEGIN
    SELECT COUNT(*) INTO v_count
      FROM variant_attribute
     WHERE variant_id = p_variant_id;
    
    IF p_position IS NULL THEN
        v_position := v_count;  -- append at end
    ELSE
        v_position := p_position;
        IF v_position < 0 THEN 
            v_position := 0;
        END IF;
        IF v_position > v_count THEN
            v_position := v_count;
        END IF;
        UPDATE variant_attribute
           SET causation_index = causation_index + 1
         WHERE variant_id = p_variant_id
           AND causation_index >= v_position;
    END IF;

    INSERT INTO variant_attribute(attribute_id, name, causation_index, variant_id)
    VALUES (p_attribute_id, p_name, v_position, p_variant_id);
END;
$$;

## Add Variant

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_variant(
    in_variant_name VARCHAR
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO variant (name)
    VALUES (in_variant_name);
END;
$$;

## Add Entity

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_entity(
    in_name VARCHAR(255),
    in_variant_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO entity(name, variant_id)
    VALUES (in_name, in_variant_id);
END;
$$;

## Roll Discrete Variant Attribute Value

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION roll_discrete_varattr(
    p_vavspan_attr_id INTEGER,
    p_exclude_span_id INTEGER
) 
RETURNS INTEGER
LANGUAGE plpgsql
AS $$
DECLARE
    v_span_id INTEGER;
    v_max_double constant double precision := 1e308;
    v_variant_attribute_id INTEGER;
    v_variant_attr_span_id INTEGER;
BEGIN
    -- Retrieve the underlying variant attribute and its associated variant_attr_span.
    SELECT variant_attribute_id, variant_attr_span_id
      INTO v_variant_attribute_id, v_variant_attr_span_id
      FROM vavspan_attr
     WHERE id = p_vavspan_attr_id;
    
    WITH
    BaseSpans AS (
         SELECT s.id AS span_id, 
                va.id AS variant_attribute_id, 
                vas.id AS variant_attr_span_id, 
                s.weight AS base_weight
         FROM variant_attribute va
         JOIN span s ON s.attribute_id = va.attribute_id
         LEFT JOIN variant_attr_span vas 
           ON vas.variant_attribute_id = va.id 
          AND vas.span_id = s.id
         WHERE va.id = v_variant_attribute_id
           AND s.type = 'discrete'
           AND (p_exclude_span_id = 0 OR s.id <> p_exclude_span_id)
    ), 
    ActiveVariations AS (
         SELECT v.id AS variation_id, 
                vav.variant_attribute_id
         FROM variation v
         JOIN vavspan_attr vav 
           ON vav.id = v.to_modify_vavspan_attr_id 
          AND vav.variant_attribute_id = v_variant_attribute_id
         WHERE v.is_inactive = false
           AND vav.id = p_vavspan_attr_id
    ), 
    InactiveSpans AS (
         SELECT DISTINCT vis.span_id
         FROM variation_inactive_span vis
         JOIN ActiveVariations av 
           ON av.variation_id = vis.variation_id
    ), 
    ActivatedSpans AS (
         SELECT DISTINCT 
                vas.span_id, 
                av.variant_attribute_id, 
                NULL::INTEGER AS variant_attr_span_id, 
                0.0 AS base_weight
         FROM variation_activated_span vas
         JOIN ActiveVariations av 
           ON av.variation_id = vas.variation_id
    ), 
    DeltaWeights AS (
         SELECT vdw.span_id, 
                SUM(vdw.delta_weight) AS total_delta
         FROM variation_delta_weight vdw
         JOIN ActiveVariations av 
           ON av.variation_id = vdw.variation_id
         GROUP BY vdw.span_id
    ), 
    AllRelevantSpans AS (
         SELECT b.span_id, b.variant_attribute_id, b.variant_attr_span_id, b.base_weight
         FROM BaseSpans b
         WHERE b.span_id NOT IN (SELECT span_id FROM InactiveSpans)
         UNION
         SELECT a.span_id, a.variant_attribute_id, a.variant_attr_span_id, a.base_weight
         FROM ActivatedSpans a
         WHERE a.span_id NOT IN (SELECT span_id FROM InactiveSpans)
    ), 
    FinalSpans AS (
         SELECT ars.span_id, 
                ars.variant_attribute_id, 
                ars.variant_attr_span_id,
                COALESCE(ars.base_weight, 0) + COALESCE(dw.total_delta, 0) AS effective_weight
         FROM AllRelevantSpans ars
         LEFT JOIN DeltaWeights dw 
           ON dw.span_id = ars.span_id
    ),
    SpanCounts AS (
         SELECT ns.num_spans, 
                SUM(fs.effective_weight / ns.num_spans) AS total_weight
         FROM FinalSpans fs
         CROSS JOIN (
             SELECT COUNT(*)::double precision AS num_spans 
             FROM FinalSpans 
             WHERE effective_weight > 0
         ) ns
         WHERE fs.effective_weight > 0
         GROUP BY ns.num_spans
    ),
    AdjustedSpans AS (
         SELECT fs.span_id,
                fs.variant_attribute_id,
                fs.variant_attr_span_id,
                fs.effective_weight,
                sc.num_spans,
                sc.total_weight,
                CASE 
                  WHEN fs.effective_weight < 1 
                    THEN ceil((fs.effective_weight * v_max_double) / (sc.total_weight * sc.num_spans))
                  ELSE ceil((fs.effective_weight / (sc.total_weight * sc.num_spans)) * v_max_double)
                END AS adjusted_weight
         FROM FinalSpans fs
         CROSS JOIN SpanCounts sc
         WHERE fs.effective_weight > 0
    ),
    Running AS (
         SELECT span_id,
                variant_attribute_id,
                variant_attr_span_id,
                adjusted_weight,
                SUM(adjusted_weight) OVER (ORDER BY span_id) AS running_total
         FROM AdjustedSpans
    ),
    OrderedSpansCTE AS (
         SELECT span_id,
                variant_attribute_id,
                variant_attr_span_id,
                adjusted_weight AS contextual_weight,
                running_total,
                LAG(running_total, 1, 0) OVER (ORDER BY span_id) AS prev_running_total
         FROM Running
    ),
    TotalAdjusted AS (
         SELECT MAX(running_total) AS v_total_weight
         FROM OrderedSpansCTE
    ),
    Rng AS (
         SELECT t.v_total_weight,
                floor(random() * t.v_total_weight) AS v_random_pick
         FROM TotalAdjusted t
    )
    SELECT o.span_id
      INTO v_span_id
      FROM OrderedSpansCTE o, Rng
      WHERE Rng.v_random_pick >= o.prev_running_total
        AND Rng.v_random_pick < o.running_total
      LIMIT 1;

    RETURN v_span_id;
END;
$$;

## Gamma Range

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION gamma_rng(shape double precision)
RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    d double precision;
    c double precision;
    x double precision;
    v double precision;
    u double precision;
BEGIN
    IF shape < 1 THEN
        -- Use the transformation: Gamma(shape) = Gamma(shape+1)*U^(1/shape)
        RETURN gamma_rng(shape + 1) * power(random(), 1.0 / shape);
    END IF;
    d := shape - 1.0/3.0;
    c := 1.0 / sqrt(9.0 * d);
    LOOP
        -- Generate a standard normal via Box–Muller:
        x := sqrt(-2 * ln(random())) * cos(2 * 3.141592653589793 * random());
        v := 1 + c * x;
        IF v <= 0 THEN
            CONTINUE;
        END IF;
        v := v * v * v;
        u := random();
        IF u < 1 - 0.0331 * x * x * x * x THEN
            RETURN d * v;
        END IF;
        IF ln(u) < 0.5 * x * x + d * (1 - v + ln(v)) THEN
            RETURN d * v;
        END IF;
    END LOOP;
END;
$$;

## Generate Weighted Random Number

In [ ]:
%%sql

-- using a fast Marsaglia–Tsang method
CREATE OR REPLACE FUNCTION weighted_random(
    min_val double precision,                   -- Minimum of desired range.
    max_val double precision,                   -- Maximum of desired range (must be > min_val).
    mode_val double precision,                  -- Desired mode (peak) in the same units.
    skew double precision DEFAULT 0,            -- Skew adjustment; between -1 and 1 only, 0 = no skew.
    concentration double precision DEFAULT 4    -- Concentration (k); should be >2 for a proper unimodal shape.
) RETURNS double precision
LANGUAGE plpgsql
AS $$
DECLARE
    m_norm      double precision;  -- Normalized desired mode in [0,1].
    alpha       double precision;
    beta        double precision;
    x_val       double precision;
    y_val       double precision;
    beta_sample double precision;
BEGIN
    IF min_val >= max_val THEN
        RAISE EXCEPTION 'min_val must be less than max_val';
    END IF;
    IF concentration < 0 THEN
        RAISE EXCEPTION 'concentration must be non-negative';
    END IF;
    IF concentration <= 2 THEN
        -- When concentration <= 2 the Beta isn't unimodal; fallback to uniform.
        RETURN min_val + random() * (max_val - min_val);
    END IF;

    -- Normalize the desired mode to [0,1].
    m_norm := (mode_val - min_val) / (max_val - min_val);

    -- Parameterize α and β so that (α-1)/(α+β-2) equals m_norm.
    alpha := m_norm * (concentration - 2) + 1;
    beta  := (1 - m_norm) * (concentration - 2) + 1;

    -- Apply skew correction if desired.
    IF skew <> 0 THEN
        alpha := alpha * (1 + skew);
        beta  := beta  * (1 - skew);
    END IF;

    -- Generate a Beta(α,β) random variable via the Gamma method.
    x_val := gamma_rng(alpha);
    y_val := gamma_rng(beta);
    beta_sample := x_val / (x_val + y_val);

    -- Scale the [0,1] Beta sample to [min_val, max_val].
    RETURN min_val + beta_sample * (max_val - min_val);
END;
$$;

## Roll Continuous Variant Attribute Value

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION roll_continuous_varattr(
    p_vavspan_attr_id INTEGER,
    p_exclude_span_id  INTEGER
)
RETURNS TABLE(chosen_span_id integer, chosen_value double precision)
LANGUAGE plpgsql
AS $$
DECLARE
    v_attribute_id         integer;
    v_decimals             integer;
    v_min                  double precision;
    v_max                  double precision;
    v_mode                 double precision;
    v_concentration        double precision;
    v_skew                 double precision;
    v_total_delta_mode     double precision := 0;
    v_total_delta_conc     double precision := 0;
    v_total_delta_skew     double precision := 0;
    v_eff_min              double precision;
    v_eff_max              double precision;
    v_eff_mode             double precision;
    v_result               double precision;
    v_clamped_result       double precision;
    v_chosen_span_id       integer;
    dummy_eff_min          double precision;
    dummy_eff_max          double precision;
    v_variant_attribute_id integer;
BEGIN
    -- Retrieve attribute details from the vavspan_attr record.
    SELECT va.attribute_id, a.decimals, a.min_value, a.max_value, a.mode_value, a.concentration, a.skew, va.id
      INTO v_attribute_id, v_decimals, v_min, v_max, v_mode, v_concentration, v_skew, v_variant_attribute_id
      FROM vavspan_attr vav
      JOIN variant_attribute va ON va.id = vav.variant_attribute_id
      JOIN attribute a ON a.id = va.attribute_id
     WHERE vav.id = p_vavspan_attr_id
     LIMIT 1;

    IF v_attribute_id IS NULL THEN
        RAISE EXCEPTION 'No matching attribute found';
    END IF;

    -- Sum variation deltas.
    SELECT COALESCE(SUM(vca.delta_mode), 0),
           COALESCE(SUM(vca.delta_conc), 0),
           COALESCE(SUM(vca.delta_skew), 0)
      INTO v_total_delta_mode, v_total_delta_conc, v_total_delta_skew
      FROM variation_continuous_attr vca
      JOIN (
            SELECT v.id
              FROM variation v
              JOIN vavspan_attr vav ON vav.id = v.to_modify_vavspan_attr_id
             WHERE vav.variant_attribute_id = v_variant_attribute_id
               AND vav.id = p_vavspan_attr_id
               AND v.is_inactive = false
           ) av ON av.id = vca.variation_id;

    -- Compute effective min, max, and mode.
    IF (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0) THEN
        v_eff_min  := (v_min + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_max  := (v_max + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
        v_eff_mode := (v_mode + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew);
    ELSE
        v_eff_min  := v_min + v_total_delta_mode;
        v_eff_max  := v_max + v_total_delta_mode;
        v_eff_mode := v_mode + v_total_delta_mode;
    END IF;

    IF v_eff_max < v_eff_min THEN
        -- Swap if needed.
        v_result  := v_eff_min;
        v_eff_min := v_eff_max;
        v_eff_max := v_result;
    END IF;

    -- Generate a weighted random value.
    v_result := weighted_random(
                   v_eff_min, 
                   v_eff_max, 
                   v_eff_mode, 
                   COALESCE(v_skew, 0),
                   COALESCE(v_concentration, 0)
               );
    v_result := round(v_result::numeric, v_decimals)::double precision;

    -- Clamp the result to the effective range.
    IF v_result < v_eff_min THEN
        v_clamped_result := v_eff_min;
    ELSIF v_result > v_eff_max THEN
        v_clamped_result := v_eff_max;
    ELSE
        v_clamped_result := v_result;
    END IF;

    -- Select the span matching the computed result.
    IF p_exclude_span_id IS NOT NULL AND p_exclude_span_id <> 0 THEN
        SELECT s.id,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN variant_attr_span vas ON vas.span_id = s.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND s.id <> p_exclude_span_id
           AND vas.id = p_vavspan_attr_id
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    ELSE
        SELECT s.id,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.min_value + v_total_delta_mode)
               END,
               CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                    THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                    ELSE (s.max_value + v_total_delta_mode)
               END
          INTO v_chosen_span_id, dummy_eff_min, dummy_eff_max
          FROM span s
          JOIN variant_attr_span vas ON vas.span_id = s.id
         WHERE s.attribute_id = v_attribute_id
           AND s.type = 'continuous'
           AND vas.id = p_vavspan_attr_id
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.min_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.min_value + v_total_delta_mode)
                END) <= v_clamped_result
           AND (CASE WHEN (v_total_delta_conc <> 0 OR v_total_delta_skew <> 0)
                     THEN (s.max_value + v_total_delta_mode) * (1 + v_total_delta_conc + v_total_delta_skew)
                     ELSE (s.max_value + v_total_delta_mode)
                END) > v_clamped_result
         LIMIT 1;
    END IF;

    chosen_span_id := v_chosen_span_id;
    chosen_value   := v_clamped_result;
    RETURN NEXT;
    RETURN;
END;
$$;

## Generate Entity State

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_state(
    IN  p_entity_id                  INTEGER,
    IN  p_time                       DOUBLE PRECISION,
    IN  p_regenerate_entity_state_id INTEGER
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_root_variant_id         INTEGER;
    v_entity_state_id         INTEGER;
    v_current_variant         INTEGER;
    v_trigger_vas             INTEGER;  -- the variant_attr_span id that triggered this variant (null for top-level)
    cur_va_id               INTEGER;
    cur_attr_type           TEXT;
    v_variant_attr_span_id  INTEGER;
    v_existing_evav_id      INTEGER;
    v_existing_span_id      INTEGER;
    v_lock_count            INTEGER := 0;
    v_new_span_id           INTEGER;
    v_new_numeric           DOUBLE PRECISION;
    v_used_span_id          INTEGER;
    v_sub_variant_id        INTEGER;
    v_attr_counter          INTEGER := 0;
    v_queue_id              INTEGER;  -- for stack (LIFO) ordering
    cur_vavspan_attr_id     INTEGER;  -- resolved unique vavspan_attr id
    rec                     RECORD;
BEGIN
    -- Get the root variant from the entity.
    SELECT variant_id
      INTO v_root_variant_id
      FROM entity
     WHERE id = p_entity_id;
    IF v_root_variant_id IS NULL THEN
       RAISE EXCEPTION 'No entity found with id=%', p_entity_id;
    END IF;

    -- Use supplied state id (for regeneration) or insert a new entity_state.
    IF p_regenerate_entity_state_id IS NOT NULL THEN
        v_entity_state_id := p_regenerate_entity_state_id;
    ELSE
        INSERT INTO entity_state(entity_id, "time")
        VALUES (p_entity_id, p_time)
        RETURNING id INTO v_entity_state_id;
    END IF;

    -- Create a temporary queue that holds variants to process.
    DROP TABLE IF EXISTS _variant_queue;
    CREATE TEMPORARY TABLE _variant_queue (
        queue_id serial PRIMARY KEY,
        variant_id INTEGER,
        variant_attr_span_id INTEGER
    ) ON COMMIT DROP;
    -- Top-level variant: no triggering variant_attr_span.
    INSERT INTO _variant_queue (variant_id, variant_attr_span_id)
      VALUES (v_root_variant_id, NULL);

    -- Temporary table to hold the current variant's attributes.
    DROP TABLE IF EXISTS _variant_attributes;
    CREATE TEMPORARY TABLE _variant_attributes (
        va_id    INTEGER,
        attr_type TEXT
    ) ON COMMIT DROP;

    -- Process the variant queue (LIFO for depth-first traversal).
    LOOP
        SELECT queue_id, variant_id, variant_attr_span_id
          INTO v_queue_id, v_current_variant, v_trigger_vas
          FROM _variant_queue
         ORDER BY queue_id DESC
         LIMIT 1;
        EXIT WHEN NOT FOUND;
        DELETE FROM _variant_queue WHERE queue_id = v_queue_id;

        -- Load the attributes for the current variant.
        TRUNCATE _variant_attributes;
        INSERT INTO _variant_attributes (va_id, attr_type)
          SELECT va.id, a.type
            FROM variant_attribute va
            JOIN attribute a ON a.id = va.attribute_id
           WHERE va.variant_id = v_current_variant;

        v_attr_counter := 0;
        FOR rec IN
            SELECT va_id, attr_type FROM _variant_attributes
        LOOP
            cur_va_id     := rec.va_id;
            cur_attr_type := rec.attr_type;
            v_attr_counter := v_attr_counter + 1;

            -- Determine the variant_attr_span ID for this attribute.
            IF v_trigger_vas IS NOT NULL THEN
                SELECT id
                  INTO v_variant_attr_span_id
                  FROM variant_attr_span
                 WHERE id = v_trigger_vas
                   AND variant_attribute_id = cur_va_id
                 LIMIT 1;
            ELSE
                v_variant_attr_span_id := NULL;
            END IF;

            -- Resolve (or create) the unique vavspan_attr record for this combination.
            SELECT id
              INTO cur_vavspan_attr_id
              FROM vavspan_attr
             WHERE variant_attribute_id = cur_va_id
               AND ((variant_attr_span_id IS NULL AND v_variant_attr_span_id IS NULL)
                    OR variant_attr_span_id = v_variant_attr_span_id)
             LIMIT 1;
            IF NOT FOUND THEN
                INSERT INTO vavspan_attr (variant_attribute_id, variant_attr_span_id)
                VALUES (cur_va_id, v_variant_attr_span_id)
                RETURNING id INTO cur_vavspan_attr_id;
            END IF;

            -- Process or update the entity_varattr_value for this resolved vavspan_attr.
            IF p_regenerate_entity_state_id IS NOT NULL THEN
                SELECT id, span_id
                  INTO v_existing_evav_id, v_existing_span_id
                  FROM entity_varattr_value
                 WHERE entity_state_id = v_entity_state_id
                   AND vavspan_attr_id = cur_vavspan_attr_id
                 LIMIT 1;
            ELSE
                v_existing_evav_id := NULL;
                v_existing_span_id := NULL;
            END IF;

            IF p_regenerate_entity_state_id IS NOT NULL AND v_existing_evav_id IS NOT NULL THEN
                SELECT count(*) INTO v_lock_count
                  FROM evav_lock
                 WHERE locked_evav_id = v_existing_evav_id;
                IF v_lock_count > 0 THEN
                    v_used_span_id := v_existing_span_id;
                ELSE
                    IF cur_attr_type = 'discrete' THEN
                        -- Use the unique vavspan_attr in the roll function call.
                        v_new_span_id := roll_discrete_varattr(cur_vavspan_attr_id, 0);
                        UPDATE entity_varattr_value
                           SET span_id = v_new_span_id
                         WHERE id = v_existing_evav_id;
                        v_used_span_id := v_new_span_id;
                    ELSE
                        SELECT t.chosen_span_id, t.chosen_value
                          INTO v_new_span_id, v_new_numeric
                          FROM roll_continuous_varattr(cur_vavspan_attr_id, 0) t
                         LIMIT 1;
                        UPDATE entity_varattr_value
                           SET span_id = v_new_span_id,
                               numeric_value = v_new_numeric
                         WHERE id = v_existing_evav_id;
                        v_used_span_id := v_new_span_id;
                    END IF;
                END IF;
            ELSE
                IF cur_attr_type = 'discrete' THEN
                    v_new_span_id := roll_discrete_varattr(cur_vavspan_attr_id, 0);
                    INSERT INTO entity_varattr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        vavspan_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        NULL,
                        v_new_span_id,
                        cur_vavspan_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                ELSE
                    SELECT t.chosen_span_id, t.chosen_value
                      INTO v_new_span_id, v_new_numeric
                      FROM roll_continuous_varattr(cur_vavspan_attr_id, 0) t
                     LIMIT 1;
                    INSERT INTO entity_varattr_value(
                        entity_state_id,
                        numeric_value,
                        span_id,
                        vavspan_attr_id
                    ) VALUES (
                        v_entity_state_id,
                        v_new_numeric,
                        v_new_span_id,
                        cur_vavspan_attr_id
                    );
                    v_used_span_id := v_new_span_id;
                END IF;
            END IF;

            -- If the chosen span activates a sub–variant, enqueue that variant.
            -- The sub–variant is defined by the variant_attr_span that produced the used span.
            SELECT variant_id
              INTO v_sub_variant_id
              FROM variant_attr_span
             WHERE variant_attribute_id = cur_va_id
               AND id = v_used_span_id
             LIMIT 1;
            IF v_sub_variant_id IS NOT NULL AND v_sub_variant_id <> v_current_variant THEN
                INSERT INTO _variant_queue (variant_id, variant_attr_span_id)
                VALUES (v_sub_variant_id, v_used_span_id)
                ON CONFLICT DO NOTHING;
            END IF;
        END LOOP;
    END LOOP;

    DROP TABLE IF EXISTS _variant_attributes;
    DROP TABLE IF EXISTS _variant_queue;
END;
$$;

## Get Entity State Details

In [ ]:
%%sql

DROP TYPE IF EXISTS entity_state_details CASCADE;
CREATE TYPE entity_state_details AS (
  "Index"                   BIGINT,
  "Attribute"               VARCHAR(255),
  "Value"                   DOUBLE PRECISION,
  "Units"                   VARCHAR(255),
  "Label"                   VARCHAR(255),
  "Active Variations"       JSONB,
  "Caused Variant"          VARCHAR(255),
  "Locked"                  BOOLEAN
);

-- Main function: given an entity id, return one row per generated variant attribute value,
-- ordered by the effective causation order (i.e. the depth–first order in which
-- generate_entity_state processed them). The returned columns are:
--
--   state_time                -- the time stamp of the entity state
--   effective_causation_index -- the effective index (computed via depth–first traversal)
--   variant_attribute_name    -- the name of the variant attribute (from variant_attribute.name)
--   variant_name              -- the name of the variant that “owns” that attribute
--   numeric_value             -- the numeric value (if any, for continuous attributes)
--   span_label                -- the label of the chosen span (if any)
--   active_variations         -- a JSON object mapping “span addresses” (from ancestors that activated a variation)
--                               to arrays of variation details (including any sub–variant name activated)
--   sub_variant_name          -- if the chosen span activated a sub–variant, its name (else null)
--   is_locked                 -- whether this entity_varattr_value is locked
CREATE OR REPLACE FUNCTION get_entity_state_details(
    p_entity_id INT,
    p_time      DOUBLE PRECISION
)
RETURNS SETOF entity_state_details
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY
  WITH RECURSIVE 
  -- Compute a per–variant sibling order using causation_index.
  ordered_va AS (
    SELECT
      va.id AS variant_attribute_id,
      va.variant_id,
      va.attribute_id,
      va.name,
      va.causation_index,
      ROW_NUMBER() OVER (PARTITION BY va.variant_id ORDER BY va.causation_index) AS sibling_order
    FROM variant_attribute va
  ),
  --
  -- Build the processing tree while computing two “paths”:
  --
  --   * attribute_path – the flattened path using "/" as separator and not including the span label.
  --     For example: Human/Male/Irish/Hair Color
  --
  --   * json_full_address – same as attribute_path, except that if a span label exists it is appended
  --     with a colon (e.g. Human/Male/Irish/Hair Color:Red). This is used as the key in Active Variations.
  --
  ev_tree AS (
    -- Anchor: start with the entity state’s root variant.
    SELECT
      es."time" AS state_time,
      ev.id AS ev_id,
      ev.entity_state_id,
      ov.variant_attribute_id,
      v.name || '/' || ov.name AS attribute_path,
      v.name || '/' || ov.name || COALESCE(':' || s.label, '') AS json_full_address,
      v.id AS variant_id,
      v.name AS variant_name,
      ev.numeric_value,
      a.units,
      ev.span_id,
      s.label AS span_label,
      vas.variant_id AS activated_sub_variant,
      ARRAY[ ov.sibling_order ] AS proc_path
    FROM entity_state es
    JOIN entity e ON es.entity_id = e.id
    JOIN variant v ON e.variant_id = v.id
    JOIN entity_varattr_value ev ON ev.entity_state_id = es.id
    JOIN ordered_va ov ON ov.variant_attribute_id = ev.variant_attribute_id
    JOIN attribute a ON a.id = ov.attribute_id
    LEFT JOIN span s ON ev.span_id = s.id
    LEFT JOIN variant_attr_span vas 
           ON vas.id = ev.span_id
          AND vas.variant_attribute_id = ov.variant_attribute_id
    WHERE e.id = p_entity_id
      AND es."time" = p_time
      AND ov.variant_id = v.id

    UNION ALL

    -- Recursive step: for any attribute that activated a sub–variant,
    -- process the sub–variant’s attributes.
    SELECT
      parent.state_time,
      ev.id AS ev_id,
      ev.entity_state_id,
      ov.variant_attribute_id,
      parent.attribute_path || '/' || ov.name AS attribute_path,
      parent.json_full_address || '/' || ov.name || COALESCE(':' || s.label, '') AS json_full_address,
      v_child.id AS variant_id,
      v_child.name AS variant_name,
      ev.numeric_value,
      a.units,
      ev.span_id,
      s.label AS span_label,
      vas.variant_id AS activated_sub_variant,
      parent.proc_path || ov.sibling_order AS proc_path
    FROM ev_tree parent
    JOIN entity_varattr_value ev ON ev.entity_state_id = parent.entity_state_id
    JOIN ordered_va ov ON ov.variant_attribute_id = ev.variant_attribute_id
    JOIN variant v_child ON ov.variant_id = v_child.id
    JOIN attribute a ON a.id = ov.attribute_id
    LEFT JOIN span s ON ev.span_id = s.id
    LEFT JOIN variant_attr_span vas 
           ON vas.id = ev.span_id
          AND vas.variant_attribute_id = ov.variant_attribute_id
    WHERE parent.activated_sub_variant IS NOT NULL
      AND ov.variant_id = parent.activated_sub_variant
  ),
  --
  -- Index the tree by ordering lexicographically on proc_path.
  ev_indexed AS (
    SELECT ev.*,
           row_number() OVER (ORDER BY ev.proc_path) AS effective_causation_index
    FROM ev_tree ev
  )
  --
  -- Final select: return one row per processed attribute value.
  SELECT 
    ev.effective_causation_index::bigint AS "Index",
    ev.attribute_path::varchar(255) AS "Attribute",
    ev.numeric_value::double precision AS "Value",
    ev.units::varchar(255) AS "Units",
    ev.span_label::varchar(255) AS "Label",
    (
      SELECT COALESCE(
        jsonb_object_agg(anc_variations.json_full_address, anc_variations.variations_array),
        '{}'::jsonb
      )
      FROM (
        SELECT
          anc.json_full_address,
          jsonb_agg(
            jsonb_build_object(
              'variation_id', v.id,
              'is_inactive', v.is_inactive,
              'variation_continuous_attr', CASE WHEN vca.id IS NOT NULL THEN to_jsonb(vca) ELSE NULL END,
              'variation_activated_span', CASE WHEN vas2.id IS NOT NULL THEN to_jsonb(vas2) ELSE NULL END,
              'variation_delta_weight', CASE WHEN vd.id IS NOT NULL THEN to_jsonb(vd) ELSE NULL END,
              'variation_inactive_span', CASE WHEN vi.id IS NOT NULL THEN to_jsonb(vi) ELSE NULL END
            )
          ) AS variations_array
        FROM ev_indexed anc
        JOIN variation v ON v.activating_span_id = anc.span_id
        LEFT JOIN variation_continuous_attr vca ON vca.variation_id = v.id
        LEFT JOIN variation_activated_span vas2 ON vas2.variation_id = v.id
        LEFT JOIN variation_delta_weight vd ON vd.variation_id = v.id
        LEFT JOIN variation_inactive_span vi ON vi.variation_id = v.id
        WHERE anc.effective_causation_index < ev.effective_causation_index
          AND anc.span_id IS NOT NULL
        GROUP BY anc.json_full_address
      ) anc_variations
    ) AS "Active Variations",
    (
      CASE 
        WHEN ev.activated_sub_variant IS NOT NULL THEN
           (SELECT v2.name FROM variant v2 WHERE v2.id = ev.activated_sub_variant)
        ELSE NULL
      END
    )::varchar(255) AS "Caused Variant",
    EXISTS (
      SELECT 1 
      FROM evav_lock 
      WHERE locked_evav_id = ev.ev_id
    ) AS "Locked"
  FROM ev_indexed ev
  ORDER BY ev.effective_causation_index;
END;
$$;

## Generate Entity Group

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE generate_entity_group(
    p_variant_id           INTEGER,
    p_group_name           VARCHAR,
    p_entity_name_template VARCHAR,
    p_num_entities         INTEGER
)
LANGUAGE plpgsql AS $$
DECLARE
    v_entity_group_id  INTEGER;
    v_entity_id        INTEGER;
    v_entity_state_id  INTEGER;
    v_idx              INTEGER;
    v_entity_name      VARCHAR(255);
BEGIN
    INSERT INTO entity_group (name)
    VALUES (p_group_name)
    RETURNING id INTO v_entity_group_id;

    FOR v_idx IN 1..p_num_entities LOOP
        v_entity_name := format(p_entity_name_template, v_idx);

        INSERT INTO entity (variant_id, name)
        VALUES (p_variant_id, v_entity_name)
        RETURNING id INTO v_entity_id;

        INSERT INTO entity_group_link (entity_id, entity_group_id)
        VALUES (v_entity_id, v_entity_group_id);

        CALL generate_entity_state(v_entity_id, 0, NULL);
    END LOOP;
END;
$$;

## Get Entity Group Details

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_entity_group_state_details(
    p_entity_group_id INT
)
RETURNS TEXT
LANGUAGE sql
AS $$
  SELECT jsonb_pretty(
           COALESCE(
             jsonb_object_agg(e.name, details),
             '{}'::jsonb
           )
         )
  FROM (
    SELECT e.name,
           (
             SELECT jsonb_agg(
                      jsonb_strip_nulls(
                        jsonb_build_object(
                          'Attribute', d."Attribute",
                          'Value', CASE 
                                     WHEN d."Value" IS NOT NULL 
                                       THEN CASE 
                                              WHEN d."Units" IS NOT NULL 
                                                THEN d."Value"::text || ' ' || d."Units"
                                              ELSE d."Value"::text
                                            END
                                     ELSE NULL
                                   END,
                          'Label', d."Label",
                          'Active Variations', CASE 
                                                 WHEN d."Active Variations" = '{}'::jsonb THEN NULL 
                                                 ELSE d."Active Variations" 
                                               END,
                          'Caused Variant', d."Caused Variant",
                          'Locked', CASE WHEN d."Locked" THEN d."Locked" ELSE NULL END
                        )
                      )
                    )
             FROM get_entity_state_details(e.id, 0) AS d
           ) AS details
    FROM entity e
    JOIN entity_group_link egl ON e.id = egl.entity_id
    WHERE egl.entity_group_id = p_entity_group_id
  ) e;
$$;

## Set Span as Sub-Variant Activating Span

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE set_variant_attr_span(
    p_variant_id INT,
    p_variant_attribute_id INT,
    p_span_id INT,
    OUT new_variant_attr_span_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO variant_attr_span(variant_id, variant_attribute_id, span_id)
  VALUES (p_variant_id, p_variant_attribute_id, p_span_id)
  RETURNING id INTO new_variant_attr_span_id;
END;
$$;

## Add Variation

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE add_variation(
    p_activating_span_id INT,
    p_activating_variant_attr_span_id INT,
    p_activating_variant_attribute_id INT,
    p_to_modify_variant_attr_span_id INT,
    p_to_modify_variant_attribute_id INT
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_to_modify_vavspan_attr_id INT;
    v_activating_vavspan_attr_id INT;
BEGIN
    -- Try to get an existing vavspan_attr row for the "to modify" attribute.
    SELECT id INTO v_to_modify_vavspan_attr_id
      FROM vavspan_attr
     WHERE variant_attribute_id = p_to_modify_variant_attribute_id
       AND variant_attr_span_id = p_to_modify_variant_attr_span_id
     LIMIT 1;

    -- If none exists, insert one.
    IF v_to_modify_vavspan_attr_id IS NULL THEN
        INSERT INTO vavspan_attr (variant_attribute_id, variant_attr_span_id)
        VALUES (p_to_modify_variant_attribute_id, p_to_modify_variant_attr_span_id)
        RETURNING id INTO v_to_modify_vavspan_attr_id;
    END IF;

    -- Try to get an existing vavspan_attr row for the "activating" attribute.
    SELECT id INTO v_activating_vavspan_attr_id
      FROM vavspan_attr
     WHERE variant_attribute_id = p_activating_variant_attribute_id
       AND variant_attr_span_id = p_activating_variant_attr_span_id
     LIMIT 1;

    -- If none exists, insert one.
    IF v_activating_vavspan_attr_id IS NULL THEN
        INSERT INTO vavspan_attr (variant_attribute_id, variant_attr_span_id)
        VALUES (p_activating_variant_attribute_id, p_activating_variant_attr_span_id)
        RETURNING id INTO v_activating_vavspan_attr_id;
    END IF;

    -- Insert the new variation using the determined vavspan_attr IDs.
    INSERT INTO variation(
        activating_span_id, 
        to_modify_vavspan_attr_id, 
        activating_vavspan_attr_id
    )
    VALUES (
        p_activating_span_id, 
        v_to_modify_vavspan_attr_id, 
        v_activating_vavspan_attr_id
    )
    RETURNING id INTO new_variation_id;
END;
$$;

## Add Parameter Delta Effect to Variation

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE set_continuous_variation_span(
    p_variation_id INT,
    p_delta_mode DOUBLE PRECISION,
    p_delta_conc DOUBLE PRECISION,
    p_delta_skew DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO variation_continuous_attr(variation_id, delta_mode, delta_conc, delta_skew)
  VALUES (p_variation_id, p_delta_mode, p_delta_conc, p_delta_skew);
END;
$$;

## Add Additional Possibility Effect to Variation

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE set_span_activating_span(
    p_variation_id INT,
    p_span_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO variation_activated_span(variation_id, span_id)
  VALUES (p_variation_id, p_span_id);
END;
$$;

## Add Delta Weight Effect to Variation

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE set_delta_weight_span(
    p_variation_id INT,
    p_span_id INT,
    p_delta_weight DOUBLE PRECISION
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO variation_delta_weight(variation_id, span_id, delta_weight)
  VALUES (p_variation_id, p_span_id, p_delta_weight);
END;
$$;

## Add Removed Possibility Effect to Variation

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE set_span_disabling_span(
    p_variation_id INT,
    p_span_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO variation_inactive_span(variation_id, span_id)
  VALUES (p_variation_id, p_span_id);
END;
$$;

## Add Forward Possibility Link

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE link_vavspan_attr(
    p_variant_attribute_id INT,
    p_variant_attr_span_id INT
)
LANGUAGE plpgsql
AS $$
BEGIN
  INSERT INTO vavspan_attr(variant_attribute_id, variant_attr_span_id)
  VALUES (p_variant_attribute_id, p_variant_attr_span_id);
END;
$$;

# Inspect Schema

## List Tables

In [ ]:
%%sql

SELECT tablename AS name
FROM pg_catalog.pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY schemaname, tablename;

## List Procedures

In [ ]:
%%sql

SELECT proname AS name, pg_catalog.pg_get_function_arguments(p.oid) AS arguments
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'p' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

## List Functions

In [ ]:
%%sql

SELECT proname AS name,
       pg_catalog.pg_get_function_arguments(p.oid) AS arguments,
       pg_catalog.pg_get_function_result(p.oid) AS return_type
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'f' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

## List Indices

In [ ]:
%%sql

SELECT i.relname AS index_name, t.relname AS table_name
FROM pg_catalog.pg_index ix
JOIN pg_catalog.pg_class i ON i.oid = ix.indexrelid
JOIN pg_catalog.pg_class t ON t.oid = ix.indrelid
JOIN pg_catalog.pg_namespace n ON n.oid = t.relnamespace
WHERE n.nspname = 'public'
ORDER BY n.nspname, t.relname, i.relname;